## Introduction:
For this project I decided I wanted to explore the restaurant scene in places I've lived in the past. I decided to explore the cities of Austin, Houston, and San Antonio to get a better glimpse into potential suggestions for investors and entrepreneurs looking to make a sound investment in restauranteering. I was looking for what type of restaurant would be the best and in which city to focus on when scouting for potential property.

The question we attempted to address by conducting this project was to discern how accurately we can predict the amount of "likes" a new restaurant opening in this region can expect to have based on the type of cuisine it will serve and which city in TexaS it will open in. For this project I analyzed and modeled the data using machine learning by comparing both linear and logistic regressions to see which method yielded better predictive capabilities after training and testing

## Data:
Once this is complete, we finally name the dataframe 'raw_dataset' as it is the most complete compiled form before needing any processing for analysis via machine learning.

First we retrieved the geographical coordinates of the three cities (Austin, Houston, and San Antonio). We then went ahead and leveraged the Foursquare API in order to obtain the URLs that gave us the raw data in JSON form. Each respective URL was then scraped for the columns: 'name', 'categories', 'latitude', 'longitude', and'id' for each city. The city column will help us when separating where the restaurants are from.

For this project, I decided to focus on those restaurants found within a 1000km radius from the coordinates that were provided by the geolocator. The Foursquare API provides us with more venue categories than we need, and therefore we had to make sure to clean our results by removing non-restaurant rows. Pulling the 'likes' data is necessary for us to make our final decisions. We don't want to be pulling information that will be discarded anyways and is of no valuable for our analysis.

We used the 'id' column in order to pull the 'likes' using the API and append the information into the dataframe. We concluded by naming the dataframe 'raw_dataset', which we used in the machine learning portion of the project.

## Methodology:
Both linear and logistic regression were used to train and test the data. Linear regression was used to predict the number of 'likes' a new restaurant in this region will acquire. Sci-Kit Learn was employed for this stage.

Logistic regression was used as the classification method. Since, we used binning when classifying by number of 'likes', we essentially made use of multinomial logistic regression to perform our analysis. Although the ranges are discrete categories, they can be considered ordinal in nature. The logistic regression will need to be specified as being both multinomial and ordinal. The Sci-Kit Learn package is perfect for this.

In [28]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

! pip install folium
import folium 
import matplotlib.pyplot as plt
import pylab as pl
import itertools
import warnings
warnings.filterwarnings('ignore')

from urllib.request import urlopen
from bs4 import BeautifulSoup

!pip install geopy
from geopy.geocoders import Nominatim 

!pip install tensorflow-tensorboard==1.5.1

from sklearn import linear_model
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, r2_score

print('All libraries have been imported')

     |████████████████████████████████| 3.0 MB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 889 kB 40.6 MB/s eta 0:00:01
  Created wheel for html5lib: filename=html5lib-0.9999999-py3-none-any.whl size=107220 sha256=7db68c8c5f9cc7a5574d5a280d2a4e729b71f895484dee25b53bf0b9b56d87ce
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/53/8d/2f/a9d4c8e58cffde4e60ba7f8aac7495de9bc12cffdc2f947723
Successfully built html5lib
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.1
    Uninstalling html5lib-1.1:
      Successfully uninstalled html5lib-1.1
  Attempting uninstall: bleach
    Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
All libraries have been imported


## Foursqaure Credentials

In [25]:
CLIENT_ID = '4PCMOK5RFRRFIESNP3Q0GQA2M00QQUEKZFVOUQYHSGWNEQU0' # your Foursquare ID
CLIENT_SECRET = 'SG4MIWLQLSAMYL0ESO2A0AEGHEATDKXBIESK4QHHF5B3USAM' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URLs
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)


# create URLs
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)


# create URLs
url3 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude3, 
    longitude3, 
    radius, 
    LIMIT)

print(url1, url2, url3)

Your credentails:
CLIENT_ID: 4PCMOK5RFRRFIESNP3Q0GQA2M00QQUEKZFVOUQYHSGWNEQU0
CLIENT_SECRET:SG4MIWLQLSAMYL0ESO2A0AEGHEATDKXBIESK4QHHF5B3USAM
https://api.foursquare.com/v2/venues/explore?&client_id=4PCMOK5RFRRFIESNP3Q0GQA2M00QQUEKZFVOUQYHSGWNEQU0&client_secret=SG4MIWLQLSAMYL0ESO2A0AEGHEATDKXBIESK4QHHF5B3USAM&v=20180604&ll=37.8044557,-122.2713563&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=4PCMOK5RFRRFIESNP3Q0GQA2M00QQUEKZFVOUQYHSGWNEQU0&client_secret=SG4MIWLQLSAMYL0ESO2A0AEGHEATDKXBIESK4QHHF5B3USAM&v=20180604&ll=37.8314089,-122.2865266&radius=1000&limit=100 https://api.foursquare.com/v2/venues/explore?&client_id=4PCMOK5RFRRFIESNP3Q0GQA2M00QQUEKZFVOUQYHSGWNEQU0&client_secret=SG4MIWLQLSAMYL0ESO2A0AEGHEATDKXBIESK4QHHF5B3USAM&v=20180604&ll=32.7174209,-117.1627714&radius=1000&limit=100


## Retrieving Foursquare City Coordinates:

In [29]:
address1 = 'Austin, Texas'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address1, latitude1, longitude1))

address2 = 'Houston, Texas'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address2, latitude2, longitude2))

address3 = 'San Antonio, Texas'

geolocator = Nominatim(user_agent="foursquare_agent")
location3 = geolocator.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address3, latitude3, longitude3))

The geograpical coordinate of Austin, Texas are 30.2711286, -97.7436995.
The geograpical coordinate of Houston, Texas are 29.7589382, -95.3676974.
The geograpical coordinate of San Antonio, Texas are 29.4246002, -98.4951405.


## Data Exploration:

In [30]:
# scrape the data from the generated URLs

results1 = requests.get(url1).json()
results1

results2 = requests.get(url2).json()
results2

results3 = requests.get(url3).json()
results3

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# FIRST CITY   

venues1 = results1['response']['groups'][0]['items']
nearby_venues1 = pd.json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues1 = nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

# SECOND CITY

venues2 = results2['response']['groups'][0]['items']
nearby_venues2 = pd.json_normalize(venues2) # flatten JSON

# filter columns
filtered_columns2 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues2 = nearby_venues2.loc[:, filtered_columns2]

# filter the category for each row
nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

# clean columns
nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

# THIRD CITY

venues3 = results3['response']['groups'][0]['items']
nearby_venues3 = pd.json_normalize(venues3) # flatten JSON

# filter columns
filtered_columns3 = ['venue.name', 'venue.categories', 'venue.location.lat', 
                    'venue.location.lng', 'venue.id']
nearby_venues3 = nearby_venues3.loc[:, filtered_columns3]

# filter the category for each row
nearby_venues3['venue.categories'] = nearby_venues3.apply(get_category_type, axis=1)

# clean columns
nearby_venues3.columns = [col.split(".")[-1] for col in nearby_venues3.columns]





print('{} venues for Austin, Texas were returned by Foursquare.'.format(nearby_venues1.shape[0]))
print()
print('{} venues for Houston, Texas were returned by Foursquare.'.format(nearby_venues2.shape[0]))
print()
print('{} venues for San Antonio, Texas were returned by Foursquare.'.format(nearby_venues3.shape[0]))

100 venues for Austin, Texas were returned by Foursquare.

100 venues for Houston, Texas were returned by Foursquare.

100 venues for San Antonio, Texas were returned by Foursquare.


In [31]:
# add locations data to the data sets of each city

nearby_venues1['city'] = 'Austin'
nearby_venues2['city'] = 'Houston'
nearby_venues3['city'] = 'San Antonio'

In [32]:
# combine the three cities into one data set

nearby_venues = nearby_venues1.copy()
nearby_venues = nearby_venues.append(nearby_venues2)
nearby_venues = nearby_venues.append(nearby_venues3)

In [33]:
nearby_venues

,name,categories,lat,lng,id,city
0,Oaklandish,Clothing Store,37.805075,-122.270726,4dfb9c2c1f6eeef806ab898c,Austin
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Austin
2,Bar Shiru,Bar,37.806378,-122.270393,5c5b9abdf870fd002c35d291,Austin
3,Cafe Van Kleef,Bar,37.806660,-122.270273,46884818f964a52056481fe3,Austin
4,Woods Bar & Brewery,Brewery,37.806889,-122.270415,5419f32c498e561ee5c2fa38,Austin
5,Cape & Cowl,Comic Shop,37.806725,-122.272747,56562410498ea43ab630819a,Austin
6,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Austin
7,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Austin
8,Lucky Duck Bicycle Cafe,Café,37.801684,-122.268656,57bf810c498ee0a34d9f8ca1,Austin
9,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Austin


In [34]:
nearby_venues['categories'].unique()

array(['Clothing Store', 'Vegetarian / Vegan Restaurant', 'Bar',
       'Brewery', 'Comic Shop', 'Bagel Shop', 'Japanese Restaurant',
       'Café', 'Vietnamese Restaurant', 'Coffee Shop', 'Music Venue',
       'Tiki Bar', 'Sandwich Place', 'Yoga Studio', 'Wine Bar',
       'Mexican Restaurant', 'Seafood Restaurant', 'Cupcake Shop',
       'Fried Chicken Joint', 'Caribbean Restaurant',
       'Chinese Restaurant', 'Dance Studio', 'Skating Rink', 'Nightclub',
       'Cocktail Bar', 'Beer Bar', 'Food Court', 'Gym / Fitness Center',
       'Hot Dog Joint', 'Bubble Tea Shop', 'Taco Place', 'Ice Cream Shop',
       'Brazilian Restaurant', 'Burger Joint', 'Climbing Gym',
       'Farmers Market', 'American Restaurant', 'Dessert Shop',
       'Thai Restaurant', 'Bakery', 'New American Restaurant', 'Gay Bar',
       'Hotpot Restaurant', 'Tea Room', 'Beer Garden',
       'Dumpling Restaurant', 'Breakfast Spot', 'Grocery Store',
       'Dim Sum Restaurant', 'Arts & Crafts Store', 'Museum',
      

In [35]:
# check list and manually remove all non-restaurant data

nearby_venues['categories'].unique()

removal_list = ['Clothing Store','Bar','Brewery', 
                'Comic Shop', 'Yoga Studio','Café', 
                'Coffee Shop', 'Tiki Bar', 'Music Venue', 
                'Wine Bar',  'Cocktail Bar', 'Dance Studio', 
                'Gym / Fitness Center','Beer Bar', 
                'Bubble Tea Shop', 'Nightclub', 'Food Court', 
                'Ice Cream Shop', 'Cupcake Shop', 'Skating Rink', 
                'Dessert Shop', 'Climbing Gym', 'Bakery', 
                'Farmers Market', 'Gay Bar','Beer Garden',
                'Tea Room','Arts & Crafts Store', 'Grocery Store', 
                'Sports Bar', 'Museum', 'Street Food Gathering', 
                'Library', 'Skate Park', 'Movie Theater','Park', 
                'Gym', 'Stadium', 'Furniture / Home Store', 'Discount Store', 
                'Playground', 'Cosmetics Shop', 'Casino', 
                'Pet Store','Electronics Store', 'Snack Place',
                'Salon / Barbershop', 'Shopping Plaza', 'Deli / Bodega', 
                'Candy Store', 'Liquor Store', 'Hotel', 
                'Shoe Store', 'Bookstore', 'Shopping Mall', 
                'Dive Bar', 'Video Game Store', 'Pharmacy', 
                'Accessories Store', 'Lingerie Store', 'Mobile Phone Shop', 
                'Pool Hall', 'Juice Bar', 'Kids Store', 
                'Supplement Shop', 'Big Box Store', 'Mattress Store', 
                'Hardware Store', 'Paper / Office Supplies Store', 'Theater', 
                'Business Service', 'Donut Shop', 'Beer Store', 
                'Lounge', 'Health Food Store', 'Pedestrian Plaza', 
                'Hookah Bar', 'Concert Hall', 'Chocolate Shop', 
                'Hostel', 'Convenience Store', 'Pub', 
                'Plaza', 'Comedy Club', 'Speakeasy', 
                'Tattoo Parlor', 'Massage Studio']

nearby_venues = nearby_venues[~nearby_venues['categories'].isin(removal_list)]

nearby_venues['categories'].unique().tolist()

['Vegetarian / Vegan Restaurant',
 'Bagel Shop',
 'Japanese Restaurant',
 'Vietnamese Restaurant',
 'Sandwich Place',
 'Mexican Restaurant',
 'Seafood Restaurant',
 'Fried Chicken Joint',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Hot Dog Joint',
 'Taco Place',
 'Brazilian Restaurant',
 'Burger Joint',
 'American Restaurant',
 'Thai Restaurant',
 'New American Restaurant',
 'Hotpot Restaurant',
 'Dumpling Restaurant',
 'Breakfast Spot',
 'Dim Sum Restaurant',
 'Indian Restaurant',
 'Diner',
 'Mediterranean Restaurant',
 'Scandinavian Restaurant',
 'Southern / Soul Food Restaurant',
 'Asian Restaurant',
 'Pizza Place',
 'Filipino Restaurant',
 'Wings Joint',
 'Burrito Place',
 'Fast Food Restaurant',
 'French Restaurant',
 'Sushi Restaurant',
 'Falafel Restaurant',
 'Theme Restaurant',
 'Italian Restaurant',
 'Turkish Restaurant',
 'Ramen Restaurant']

## DataFrame Creation:

In [36]:
# set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


nearby_venues['likes'] = like_list
nearby_venues.head()

[77, 22, 156, 71, 14, 202, 51, 65, 8, 104, 369, 94, 33, 61, 177, 188, 39, 40, 43, 23, 56, 13, 239, 102, 11, 73, 259, 25, 33, 45, 69, 99, 230, 5, 39, 24, 52, 36, 43, 120, 55, 332, 133, 35, 24, 31, 61, 13, 92, 56, 18, 16, 22, 5, 68, 17, 65, 61, 16, 4, 1, 41, 0, 3, 31, 156, 76, 2, 9, 78, 131, 142, 26, 41, 105, 171, 19, 18, 24, 34, 94, 35, 35, 296, 30, 31, 127, 103, 104, 320, 132, 185, 480, 540, 31, 488, 174, 54, 204, 31]


,name,categories,lat,lng,id,city,likes
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Austin,77
6,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Austin,22
7,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Austin,156
9,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Austin,71
13,Anula's Cafe,Sandwich Place,37.803583,-122.270151,4b50d22df964a520a73327e3,Austin,14


In [37]:
# this is really the raw dataset now so let us rename it something more appropriate

raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,city,likes
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Austin,77
6,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Austin,22
7,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Austin,156
9,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Austin,71
13,Anula's Cafe,Sandwich Place,37.803583,-122.270151,4b50d22df964a520a73327e3,Austin,14


In [38]:
# inspecting the raw dataset shows that there may be too many different types of cuisines

raw_dataset['categories'].unique().tolist()

['Vegetarian / Vegan Restaurant',
 'Bagel Shop',
 'Japanese Restaurant',
 'Vietnamese Restaurant',
 'Sandwich Place',
 'Mexican Restaurant',
 'Seafood Restaurant',
 'Fried Chicken Joint',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 'Hot Dog Joint',
 'Taco Place',
 'Brazilian Restaurant',
 'Burger Joint',
 'American Restaurant',
 'Thai Restaurant',
 'New American Restaurant',
 'Hotpot Restaurant',
 'Dumpling Restaurant',
 'Breakfast Spot',
 'Dim Sum Restaurant',
 'Indian Restaurant',
 'Diner',
 'Mediterranean Restaurant',
 'Scandinavian Restaurant',
 'Southern / Soul Food Restaurant',
 'Asian Restaurant',
 'Pizza Place',
 'Filipino Restaurant',
 'Wings Joint',
 'Burrito Place',
 'Fast Food Restaurant',
 'French Restaurant',
 'Sushi Restaurant',
 'Falafel Restaurant',
 'Theme Restaurant',
 'Italian Restaurant',
 'Turkish Restaurant',
 'Ramen Restaurant']

In [39]:
# we can group some cuisines together to make a better categorical variable

european = ['Mediterranean Restaurant', 'Scandinavian Restaurant', 'Pizza Place',
       'French Restaurant', 'Falafel Restaurant', 'Italian Restaurant',
       'Turkish Restaurant']

latin = ['Mexican Restaurant', 'Taco Place', 'Brazilian Restaurant', 
          'Burrito Place']

asian = ['Japanese Restaurant', 'Vietnamese Restaurant', 'Chinese Restaurant',
         'Hot Dog Joint', 'Hotpot Restaurant', 'Indian Restaurant',
         'Thai Restaurant', 'Dumpling Restaurant', 'Dim Sum Restaurant',
         'Asian Restaurant', 'Filipino Restaurant', 'Sushi Restaurant',
         'Ramen Restaurant']

american = ['Vegetarian / Vegan Restaurant', 'Seafood Restaurant', 'Caribbean Restaurant',
           'Burger Joint', 'American Restaurant', 'New American Restaurant',
            'Southern / Soul Food Restaurant', 'Diner']

casual = ['Bagel Shop', 'Sandwich Place', 'Fried Chicken Joint', 
          'Breakfast Spot', 'Wings Joint', 'Fast Food Restaurant',
          'Theme Restaurant']

def conditions(s):
    if s['categories'] in european:
        return 'european'
    if s['categories'] in latin:
        return 'latin'
    if s['categories'] in asian:
        return 'asian'
    if s['categories'] in american:
        return 'american'
    if s['categories'] in casual:
        return 'casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Austin,77,american
6,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Austin,22,casual
7,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Austin,156,asian
9,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Austin,71,asian
13,Anula's Cafe,Sandwich Place,37.803583,-122.270151,4b50d22df964a520a73327e3,Austin,14,casual
16,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Austin,202,latin
17,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Austin,51,asian
18,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Austin,65,american
23,World Famous Hotboys,Fried Chicken Joint,37.806526,-122.272040,5e0a805333617d00086cd498,Austin,8,casual
24,Miss Ollie's,Caribbean Restaurant,37.801448,-122.274680,50be580af136a1fe01a5db73,Austin,104,american


In [40]:
# double check to make sure categories_classified has been created correctly

pd.crosstab(index = raw_dataset["categories_classified"],
            columns="count")

col_0,count
categories_classified,
american,19
asian,27
casual,18
european,17
latin,19


In [41]:
raw_dataset['likes'].mean()

91.36

In [42]:
# create a function to bin for us

def rankings(df):
    
    if df['likes'] <= 60:
        return 3
    
    elif df['likes'] <= 100:
        return 2
    
    elif df['likes'] > 100:
        return 1

In [43]:
# apply rankings function to dataset

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,city,likes,categories_classified,ranking
1,Golden Lotus Vegetarian Restaurant,Vegetarian / Vegan Restaurant,37.803290,-122.270473,49cebb1bf964a520785a1fe3,Austin,77,american,2
6,Beauty’s Bagel Shop,Bagel Shop,37.806082,-122.268356,5bd0959cf1fdaf002ce03e11,Austin,22,casual,3
7,Abura-Ya,Japanese Restaurant,37.805959,-122.267693,539a69a7498ee67090b2b285,Austin,156,asian,1
9,Tay Ho Restaurant & Bar,Vietnamese Restaurant,37.802062,-122.269573,4c8b16ec52a98cfad73533e9,Austin,71,asian,2
13,Anula's Cafe,Sandwich Place,37.803583,-122.270151,4b50d22df964a520a73327e3,Austin,14,casual,3
16,Cosecha,Mexican Restaurant,37.801607,-122.274889,4e179c7752b123a586cef176,Austin,202,latin,1
17,Binh Minh Quan,Vietnamese Restaurant,37.802020,-122.269396,4a75022cf964a52043e01fe3,Austin,51,asian,3
18,The Cook And Her Farmer,Seafood Restaurant,37.801583,-122.274860,5376a879498e8eeb2402cd71,Austin,65,american,2
23,World Famous Hotboys,Fried Chicken Joint,37.806526,-122.272040,5e0a805333617d00086cd498,Austin,8,casual,3
24,Miss Ollie's,Caribbean Restaurant,37.801448,-122.274680,50be580af136a1fe01a5db73,Austin,104,american,1


## Machine Learning | Logistic Regression:

In [45]:
# create dummies for linear regression modelling

# one hot encoding
reg_dataset = pd.get_dummies(raw_dataset[['categories_classified', 
                                          'city',]], 
                               prefix="", 
                               prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]


reg_dataset.head()

,name,american,asian,casual,european,latin,Austin,Houston,San Antonio,ranking,likes
1,Golden Lotus Vegetarian Restaurant,1,0,0,0,0,1,0,0,2,77
6,Beauty’s Bagel Shop,0,0,1,0,0,1,0,0,3,22
7,Abura-Ya,0,1,0,0,0,1,0,0,1,156
9,Tay Ho Restaurant & Bar,0,1,0,0,0,1,0,0,2,71
13,Anula's Cafe,0,0,1,0,0,1,0,0,3,14


In [46]:
# Multiple Linear Regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Austin', 
                         'Houston', 'San Antonio']])

y = np.asanyarray(train[['likes']])
regr.fit (x, y)

# The coefficients

print ('Coefficients: ', regr.coef_)

Coefficients:  [[ 46.38139602  -1.96544972  -7.62296229 -24.03579578 -12.75718823
   -1.10835992 -34.00759739  35.11595731]]


In [47]:
# Multiple Linear Regression Prediction Capabilities

y_hat= regr.predict(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Austin', 
                         'Houston', 'San Antonio']])

x = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Austin', 
                         'Houston', 'San Antonio']])

y = np.asanyarray(test[['likes']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

Residual sum of squares: 16023.25
Variance score: 0.01


## Machine learning | Logistic Regression

In [48]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['american', 'asian', 'casual',
                         'european', 'latin', 'Austin', 
                         'Houston', 'San Antonio']])

y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['american', 'asian', 'casual',
                         'european', 'latin', 'Austin', 
                         'Houston', 'San Antonio']])

y_test = np.asanyarray(test['ranking'])


mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[ 0.32918225 -0.17048592 -0.06377564  0.14025372 -0.23517706  0.05325494
 -0.6921135   0.6388559 ]


In [49]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob


# average = None, average = 'micro', average = 'macro', or average = 'weighted'
jaccard_score(y_test, yhat, average='weighted')

0.3302235521915156

In [50]:
log_loss(y_test, yhat_prob)

0.9695765488941727

In [52]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['american', 'asian', 'casual',
                                   'european', 'latin', 'Austin', 
                                   'Houston', 'San Antonio']])

y_all = np.asanyarray(reg_dataset['ranking'])



LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print(coef)

[ 0.4177708  -0.26541476 -0.19642341 -0.0373987   0.08147725  0.05307174
 -0.71630655  0.66324599]


In [53]:
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.67      0.36      0.47        11
           2       0.00      0.00      0.00         2
           3       0.47      0.80      0.59        10

    accuracy                           0.52        23
   macro avg       0.38      0.39      0.35        23
weighted avg       0.52      0.52      0.48        23



## Results

A linear regression model was trained on a random subsample of 80% and then the other 20% was used for testing purposes. In order to evaluate if the model is reasonable, the residual sum of squares and variance score were both calculated (16023.25, 0.01). The variance score is quite low, which means that is not a good way of modeling our data. Therefore, we moved on to logistic regression for our analysis.

The multinomial ordinal logistic regression model was also trained on a random subsample of 80% and then tested on the remaining 20%. The jaccard score and log-loss were both calculated (33.02% and .9695 respectively). Although the prediction is not promosing, a jaccard score of 33% is somewhat reasonable. The classification report is included in the analysis.

Given the modestly accurate ability of this mode, we have the ability to run the model on the complete dataset. The coefficients we got show that opening a restaurant in Austin, or serving cuisine that is asian, or casual, are negatively associated with 'likes'.

## Discussion

The first thing to note is that given the data, logistic regression presents a better fit for the data over linear regression. Using logistic regression we were able to obtain a Jaccard Score of 33.02%, which although not perfect, is more reasonable than the low variance score obtained from the linear regression. As stated before, please note that for the purposes of this project, we are assumming that likes are a good proxy for how well a new restaurant will do in terms of brand, image and by extension how well the restaurant will perform business-wise. Whether or not these assumptions hold up in a real-life scenario is up for discussion, but this project does contain limitations in scope due to the amount of data that can be fetched from the FourSquare API.

As such, to obtain insights into this data, we can proceed with breaking down the results of the logistic regression model. The results showed that the precision score for classifying whether the new restaurant would fall into classes 1, 2, or 3 (highest, medium, lowest) were 40%, 0%, and 50%. Therefore, the model is better at predicting if a restaurant will fall into the best or worst percentile of likes. This is good as we are mostly concerned with whether the restuarant will perform well or not so the high accuracy of predictions for the two extremum is a welcome feature. This allows us to fairly accurately predict the general performance of the business opportunity. Different binning methods for the classes were attempted, but the use of 3 bins by far yielded the best Jaccard Score.

Additionally, not only are we attempting to predict the general business performance but also pull insights to inform on business strategy. In this case strategy insight can be gleamed from the coefficient values from running the logistic regression on the full dataset. As such, we can see that opening a restaurant in Austin, or serving cuisine that is asian or casual in nature, are associated Positively with "likes." This suggests that the business opportunity should be opening a restaurant in either Austin, with a cuisine that is American, Casual, Asian, or Latin in nature would be the best approach for maximizing likes.

## Conclusion:

In conclusion, after analyzing restaurant 'likes' in Texas from the 300 restaurants, we can conclude that the approach to best take when looking to maximize business performance (as measured by 'likes') is to open a restaurant that is either American, Casual, Asian, or Latin and that opening the venue in Austin would be the best approach. Additionally, the predictive capabilities of the logistic regression prediction model proved to be the most accurate for classifying whether a restaurant fell in either the best or worst classes when the data was binned into their 3 respective classes.